<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC - Création du dataset "Evolution CA"

**Tags:** #fec #finance #snippet #operation #ca #revenue

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** Ce notebook permet de créer le dataset pour visualiser l'évolution du CA entre l'année N et l'année N-1.

## Input

### Import libraries

In [ ]:
import naas_data_product

### Setup Variables

In [ ]:
# Inputs
input_folder_path = "/home/ftp/FEC-engine/outputs/FEC/bdd_fin"

# Outputs
output_folder_path = "/home/ftp/FEC-engine/outputs/FEC/dataset_evolution_ca"

## Model

### Récupération du dernier fichier input

In [ ]:
df_input = get_last_df(input_folder_path)
print("✅ Row fetched:", len(df_input))
df_input.head(1)

### Création du dataset "EVOLUTION_CA"

In [ ]:
# Creation du dataset evol_ca
dataset_evol_ca = df_input.copy()

# Filtre COMPTE_NUM = Chiffre d'Affaire (RUBRIQUE N1)
dataset_evol_ca = dataset_evol_ca[
    dataset_evol_ca["COMPTE_NUM"].str.contains(r"^70|^71|^72")
]

# Regroupement
to_group = ["ENTITY", "PERIOD", "MONTH", "MONTH_INDEX", "RUBRIQUE_N3"]
to_agg = {"VALUE": "sum"}
dataset_evol_ca = dataset_evol_ca.groupby(to_group, as_index=False).agg(to_agg)

dataset_evol_ca["VALUE"] = dataset_evol_ca["VALUE"].abs()


# Calcul de la somme cumulée
dataset_evol_ca = dataset_evol_ca.sort_values(
    by=["ENTITY", "PERIOD", "MONTH_INDEX"]
).reset_index(drop=True)
dataset_evol_ca["MONTH_INDEX"] = pd.to_datetime(
    dataset_evol_ca["MONTH_INDEX"], format="%m"
).dt.strftime("%m")
dataset_evol_ca["VALUE_CUM"] = dataset_evol_ca.groupby(
    ["ENTITY", "PERIOD"], as_index=True
).agg({"VALUE": "cumsum"})


dataset_evol_ca["VALUE_D"] = (dataset_evol_ca["VALUE"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
dataset_evol_ca["VALUE_CUM_D"] = (dataset_evol_ca["VALUE_CUM"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")

# Affichage du modèle de donnée
dataset_evol_ca

## Output

### Sauvegarde des fichiers en csv

In [ ]:
save_df(dataset_evol_ca, output_folder_path)